In [ ]:
import numpy as np
import qiskit
from qoop.pcompilation.pcompilation import pcompilation
from qoop.core.ansatz import WchainCNOT_xyz
from qoop.compilation.qsp import QuantumStatePreparation
from qiskit.primitives import Sampler
from qiskit_aer import Aer
def prepare_state(v):
    num_qubits = int(np.log2(len(v)))
    qsp = QuantumStatePreparation(
        u = WchainCNOT_xyz(num_qubits, num_qubits), 
        target_state = v).fit()
    return qsp.compiler.thetass[-1]

def qmatmul(origin_L, origin_v, num_layers = 1):
    size = origin_v.shape[0]
    res = np.zeros(size)
    norm_L =  np.linalg.norm(origin_L, axis=1, keepdims=True)
    norm_v = np.linalg.norm(origin_v)
    L = origin_L / norm_L
    v = origin_v / norm_v
    num_qubits = int(np.log2(len(origin_v)))
    US = [0]*size
    V = prepare_state(v)
    for i in range(size):
        US[i] = prepare_state(L[i, :])
        sub_qc = WchainCNOT_xyz(num_qubits, num_layers)
        qc = qiskit.QuantumCircuit(1 + 2 * num_qubits, 1)
        qc.h(0)
        qc.append(sub_qc.assign_parameters(US[i]), range(1, num_qubits + 1))
        qc.append(sub_qc.assign_parameters(V), range(num_qubits + 1, 2*num_qubits + 1))
        for j in range(num_qubits):
            qc.cswap(0, j + 1, j + num_qubits + 1)
        qc.h(0)
        qc.measure(0, 0)
        simulator = Aer.get_backend('aer_simulator_statevector', device = 'GPU')
        qc = qiskit.transpile(qc, backend = simulator)
        result = simulator.run(qc).result()
        counts = result.get_counts(qc)
        res[i] = (norm_v * norm_L[i] * np.sqrt(counts.get(0,0) - counts.get(1, 0)))
    return res


num_repeats = 100
import pandas as pd
for size in [4, 8, 16, 32]:
    for num_layers in range(1, int(np.log2(size)) + 1):
        print(f"Size: {size}, Layers: {num_layers}")
        mses = []
        for _ in range(num_repeats):
            origin_L = np.random.uniform(1, 4, (size, size))
            origin_v = np.random.uniform(1, 2, size)
            res = qmatmul(origin_L, origin_v, num_layers)
            mse = (np.square(res - np.dot(origin_L, origin_v))).mean(axis=0)
            mses.append(mse)
        print(f"Mean MSE: {np.mean(mses):.4f}, Std MSE: {np.std(mses):.4f}")
        record = {'size': size, 'num_layers': num_layers, 'mean_mse': np.mean(mses), 'std_mse': np.std(mses)}
        if 'results_df' not in locals():
            results_df = pd.DataFrame(columns=record.keys())
        results_df = pd.concat([results_df, pd.DataFrame([record])], ignore_index=True)
        results_df.to_csv(f'benchmark_acc.csv', index=False)

/var/folders/bw/2cm4rnhn0djgg95n4py1pysc0000gp/T/ipykernel_37493/3283396053.py:35: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  sampler = Sampler()
/var/folders/bw/2cm4rnhn0djgg95n4py1pysc0000gp/T/ipykernel_37493/3283396053.py:38: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  res[i] = (norm_v * norm_L[i] * np.sqrt(counts.get(0,0) - counts.get(1, 0)))
/Users/vutuanhai/Documents/GitHub/QuantumNCut/qoop/core/state.py:273: UserWarning: The input state is not normalized, we will normalize it for you
  warnings.warn("The 

In [7]:
import numpy as np
import qiskit
from qoop.pcompilation.pcompilation import pcompilation
from qoop.core.ansatz import WchainCNOT_xyz
from qoop.compilation.qsp import QuantumStatePreparation
from qiskit.primitives import Sampler
from qiskit_aer import Aer
def prepare_state(v):
    num_qubits = int(np.log2(len(v)))
    qsp = QuantumStatePreparation(
        u = WchainCNOT_xyz(num_qubits, num_qubits), 
        target_state = v).fit()
    return qsp.compiler.thetass[-1]

def qmatmul(origin_L, origin_v, num_layers = 1):
    size = origin_v.shape[0]
    res = np.zeros(size)
    norm_L =  np.linalg.norm(origin_L, axis=1, keepdims=True)
    norm_v = np.linalg.norm(origin_v)
    L = origin_L / norm_L
    v = origin_v / norm_v
    num_qubits = int(np.log2(len(origin_v)))
    US = [0]*size
    V = prepare_state(v)
    for i in range(size):
        US[i] = prepare_state(L[i, :])
        sub_qc = WchainCNOT_xyz(num_qubits, num_layers)
        qc = qiskit.QuantumCircuit(1 + 2 * num_qubits, 1)
        qc.h(0)
        qc.append(sub_qc.assign_parameters(US[i]), range(1, num_qubits + 1))
        qc.append(sub_qc.assign_parameters(V), range(num_qubits + 1, 2*num_qubits + 1))
        for j in range(num_qubits):
            qc.cswap(0, j + 1, j + num_qubits + 1)
        qc.h(0)
        qc.measure(0, 0)
        simulator = Aer.get_backend('aer_simulator_statevector', device = 'GPU')
        qc = qiskit.transpile(qc, backend = simulator)
        result = simulator.run(qc).result()
        counts = result.get_counts(qc)
        print(counts)
        print(counts.get('0',0), counts.get('1', 0))
        res[i] = (norm_v * norm_L[i] * (np.sqrt(counts.get('0',0) - counts.get('1', 0)))/(np.sqrt(counts.get('0',0) + counts.get('1', 0))))
    return res
num_repeats = 1
import pandas as pd
for size in [4]:
    for num_layers in [2]:
        print(f"Size: {size}, Layers: {num_layers}")
        mses = []
        for _ in range(num_repeats):
            origin_L = np.random.uniform(1, 4, (size, size))
            origin_v = np.random.uniform(1, 2, size)
            res = qmatmul(origin_L, origin_v, num_layers)
            mse = (np.square(res - np.dot(origin_L, origin_v))).mean(axis=0)
            mses.append(mse)
        print(f"Mean MSE: {np.mean(mses):.4f}, Std MSE: {np.std(mses):.4f}")
        record = {'size': size, 'num_layers': num_layers, 'mean_mse': np.mean(mses), 'std_mse': np.std(mses)}
        if 'results_df' not in locals():
            results_df = pd.DataFrame(columns=record.keys())
        results_df = pd.concat([results_df, pd.DataFrame([record])], ignore_index=True)
        results_df.to_csv(f'benchmark_acc.csv', index=False)

Size: 4, Layers: 2


c:\Users\Public\Documents\Github\QuantumNCut\qoop\core\state.py:273: UserWarning: The input state is not normalized, we will normalize it for you
  warnings.warn("The input state is not normalized, we will normalize it for you")


{'0': 970, '1': 54}
970 54


C:\Users\haime\AppData\Local\Temp\ipykernel_41324\1488898681.py:42: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  res[i] = (norm_v * norm_L[i] * (np.sqrt(counts.get('0',0) - counts.get('1', 0)))/(np.sqrt(counts.get('0',0) + counts.get('1', 0))))


{'0': 1017, '1': 7}
1017 7


C:\Users\haime\AppData\Local\Temp\ipykernel_41324\1488898681.py:42: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  res[i] = (norm_v * norm_L[i] * (np.sqrt(counts.get('0',0) - counts.get('1', 0)))/(np.sqrt(counts.get('0',0) + counts.get('1', 0))))


{'0': 1014, '1': 10}
1014 10


C:\Users\haime\AppData\Local\Temp\ipykernel_41324\1488898681.py:42: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  res[i] = (norm_v * norm_L[i] * (np.sqrt(counts.get('0',0) - counts.get('1', 0)))/(np.sqrt(counts.get('0',0) + counts.get('1', 0))))
c:\Users\Public\Documents\Github\QuantumNCut\qoop\core\state.py:273: UserWarning: The input state is not normalized, we will normalize it for you
  warnings.warn("The input state is not normalized, we will normalize it for you")


{'0': 978, '1': 46}
978 46
Mean MSE: 0.0182, Std MSE: 0.0000


C:\Users\haime\AppData\Local\Temp\ipykernel_41324\1488898681.py:42: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  res[i] = (norm_v * norm_L[i] * (np.sqrt(counts.get('0',0) - counts.get('1', 0)))/(np.sqrt(counts.get('0',0) + counts.get('1', 0))))
